In [31]:
import os
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [57]:
import os

pretrained_path = 'bert/multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
os.environ['TF_KERAS'] = '1'

In [ ]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf-8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [54]:
def file_open(input):
    sentences1 = []
    sentences2 = []
    with open(input, 'r', encoding='utf-8') as fileobj:
        while True:
            line = fileobj.readline()
            if line :
                s = line.split('\t')
                s[0] = s[0].rstrip().replace('.', ' ').replace('"', ' ').replace("'", " ").replace("-", " ").replace(":", " ").replace("%", " ").replace("*", " ").replace("!", " ").replace("?", " ").replace("@", " ").replace('/', ' ').replace(",", ' ').replace('(', ' ').replace(')', ' ').replace('#', ' ')
                sentences1.append(s[0])
                s[1] = s[1].rstrip().replace('.', ' ').replace('"', ' ').replace("'", " ").replace("-", " ").replace(":", " ").replace("%", " ").replace("*", " ").replace("!", " ").replace("?", " ").replace("@", " ").replace('/', ' ').replace(",", ' ').replace('(', ' ').replace(')', ' ').replace('#', ' ')
                sentences2.append(s[1])
            else:
                break
    return sentences1, sentences2

In [55]:
def label_open(input):
    with open(input, "r") as fileobj:
        labels = [float(s.strip())/5. for s in fileobj.readlines()]
    labels = np.asarray(labels)
    return labels

In [56]:
import numpy as np
from keras_bert import Tokenizer

sentences1, sentences2 = file_open("input/STS.input.OnWN.txt")
labels = label_open("gs/STS.gs.OnWN.txt")

sentences1n, sentences2n = file_open("input/STS.input.MSRvid.txt")
sentences1 = np.concatenate([sentences1, sentences1n], axis=0)
sentences2 = np.concatenate([sentences2, sentences2n], axis=0)
labelsn = label_open("gs/STS.gs.MSRvid.txt")
labels = np.concatenate([labels, labelsn], axis=0)

sentences1n, sentences2n = file_open("input/STS.input.SMTeuroparl.txt")
sentences1 = np.concatenate([sentences1, sentences1n], axis=0)
sentences2 = np.concatenate([sentences2, sentences2n], axis=0)
labelsn = label_open("gs/STS.gs.SMTeuroparl.txt")
labels = np.concatenate([labels, labelsn], axis=0)


sentences1n, sentences2n = file_open("input/STS.input.tweet-news.txt")
sentences1 = np.concatenate([sentences1, sentences1n], axis=0)
sentences2 = np.concatenate([sentences2, sentences2n], axis=0)
labelsn = label_open("gs/STS.gs.tweet-news.txt")
labels = np.concatenate([labels, labelsn], axis=0)

test1, test2 = file_open("input/STS.input.images.txt")

indices = []
segments = []

tokenizer = Tokenizer(token_dict)

for text in sentences1:
    indice, segment = tokenizer.encode(first=text, max_len=75)
    indices.append(indice)
    segments.append(segment)
for text in sentences2:
    indice, segment = tokenizer.encode(first=text, max_len=75)
    indices.append(indice)
    segments.append(segment)


for text in test1:
    indice, segment = tokenizer.encode(first=text, max_len=75)
    indices.append(indice)
    segments.append(segment)
for text in test2:
    indice, segment = tokenizer.encode(first=text, max_len=75)
    indices.append(indice)
    segments.append(segment)
    
print('Shape of data tensor:', sentences1.shape)
print('Shape of labels tensor:', labels.shape)


In [ ]:
from keras_bert import load_trained_model_from_checkpoint

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=75, trainable=True)
bert.summary()

In [ ]:
from nltk.stem import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import nltk

In [ ]:
bert_predict = bert.predict([np.array(indices), np.array(segments)])

In [58]:
train_data = bert_predicts[: 5968]
test_data = bert_predicts[5968: ]
train_left = train_data[: 2984]
train_right = train_data[2984: ]
test_left = test_data[: 750]
test_right = test_data[750: ]

In [59]:
train_left.shape, train_right.shape,test_left.shape, test_right.shape

((2984, 75, 768), (2984, 75, 768), (750, 75, 768), (750, 75, 768))

In [60]:
train_left_cls = []
train_right_cls = []
test_left_cls = []
test_right_cls = []
train_left_words = []
train_right_words = []
test_left_words = []
test_right_words = []

for line_left, line_right in zip(train_left, train_right):
    train_left_cls.append(line_left[0])
    train_right_cls.append(line_right[0])
    train_left_words.append(line_left[1: ])
    train_right_words.append(line_right[1:  ])
for line_left, line_right in zip(test_left, test_right):
    test_left_cls.append(line_left[0])
    test_right_cls.append(line_right[0])
    test_left_words.append(line_left[1: ])
    test_right_words.append(line_right[1: ])
    
train_left_cls = np.array(train_left_cls)
train_right_cls = np.array(train_right_cls)
train_left_words = np.array(train_left_words)
train_right_words = np.array(train_right_words)
test_left_cls = np.array(test_left_cls)
test_right_cls = np.array(test_right_cls)
test_left_words = np.array(test_left_words)
test_right_words = np.array(test_right_words)
data_left = lemmatize(sentences1)
data_right = lemmatize(sentences2)

In [61]:
train_left_cls.shape, train_right_cls.shape, test_left_cls.shape, test_right_cls.shape,  train_left_words.shape, train_right_words.shape, test_left_words.shape, test_right_words.shape

((2984, 768),
 (2984, 768),
 (750, 768),
 (750, 768),
 (2984, 74, 768),
 (2984, 74, 768),
 (750, 74, 768),
 (750, 74, 768))

In [62]:
labels.shape

(2984,)

In [63]:
from nltk.stem import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import nltk


def lemmatize(texts):
    s = []
    a = np.zeros((len(texts), 75, 300))
    for text in texts:
        s.append(text_to_word_sequence(text))
    for i, text in enumerate(s):
        for j, word in enumerate(text):
            if word in word2vec:
                a[i][j][:] = word2vec[word]
            else:
                continue
    return a

In [78]:
max_len = 75


indices = np.arange(train_left_words.shape[0])
np.random.shuffle(indices)

data_left = data_left[indices]
data_right = data_right[indices]

train_left_words = train_left_words[indices]
train_right_words = train_right_words[indices]
train_left_cls = train_left_cls[indices]
train_right_cls = train_right_cls[indices]

labels = labels[indices]


In [132]:
import keras
from keras.layers import *
from keras.models import *
from keras_self_attention import SeqSelfAttention
max_len = 75

def build_model():
    input_left = Input(shape=(max_len, 300), name='input1')
    input_right = Input(shape=(max_len, 300), name='input2')
    input_bert_left = Input(shape=(max_len-1, 768), name='input3')
    input_bert_right = Input(shape=(max_len-1, 768), name='input4')
    input_cls_left = Input(shape=(768, ))
    input_cls_right = Input(shape=(768, ))
   

    attention = SeqSelfAttention(attention_activation='sigmoid', kernel_regularizer=keras.regularizers.l2(1e-6), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4)

    conv2 = Conv1D(300, 3, activation='relu')
    conv4 = Conv1D(300, 5, activation='relu')
    conv6 = Conv1D(300, 7, activation='relu')
    
    embed_left = attention(input_left)

    left2 = conv2(embed_left)
    left2 = MaxPooling1D(2)(left2)
    left2 = GlobalAveragePooling1D()(left2)
      
    left4 = conv4(embed_left)
    left4 = MaxPooling1D(2)(left4)
    left4 = GlobalAveragePooling1D()(left4)
      
    left6 = conv6(embed_left)
    left6 = MaxPooling1D(2)(left6)
    left6 = GlobalAveragePooling1D()(left6)
  
    all_embed_left = GlobalAveragePooling1D()(input_left)

    left2 = add([left2, all_embed_left])
    left4 = add([left4, all_embed_left])
    left6 = add([left6, all_embed_left])
   
    embed_right = attention(input_right)

    right2 = conv2(embed_right)
    right2 = MaxPooling1D(2)(right2)
    right2 = GlobalAveragePooling1D()(right2)
    
    right4 = conv4(embed_right)
    right4 = MaxPooling1D(2)(right4)
    right4 = GlobalAveragePooling1D()(right4)
    
    right6 = conv6(embed_right)
    right6 = MaxPooling1D(2)(right6)
    right6 = GlobalAveragePooling1D()(right6)
    
    all_embed_right = GlobalAveragePooling1D()(input_right)
    

    right2 = add([right2, all_embed_right])
    right4 = add([right4, all_embed_right])
    right6 = add([right6, all_embed_right])
    
    bert_left = GlobalAveragePooling1D()(input_bert_left)
    bert_right = GlobalAveragePooling1D()(input_bert_right)
    
    bert_left = add([bert_left, input_cls_left])
    bert_right = add([bert_right, input_cls_right])

    out2 = Dot(axes=-1, normalize=True)([left2, right2])
    out4 = Dot(axes=-1, normalize=True)([left4, right4])
    out6 = Dot(axes=-1, normalize=True)([left6, right6])
    
    outb = Dot(axes=-1, normalize=True)([bert_left, bert_right])
   
    out = average([out2, out4, out6, outb])

    model = Model(inputs=[input_left, input_right, input_bert_left, input_bert_right, input_cls_left, input_cls_right], output=out)

    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    model.summary()
    return model

In [ ]:
from keras.callbacks import *

model = build_model()
history = model.fit([data_left, data_right, train_left_words, train_right_words, train_left_cls, train_right_cls], labels, validation_split=0.2, epochs=20, batch_size=32, callbacks=[ModelCheckpoint(filepath='my_model_w2v+bert.h5', monitor='val_loss', save_best_only=True)])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
num_epochs=100

mae = history.history["mae"]
val_mae = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(mae) + 1)

plt.plot(epochs, loss, 'bo', label='Traing loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, mae, 'bo', label='Traing mae')
plt.plot(epochs, val_mae, 'b', label='Validation mae')
plt.title('Training and validation mae')
plt.legend()

plt.show()

In [135]:
test1, test2 = file_open("input/STS.input.images.txt")
test1 = lemmatize(test1)
test2 = lemmatize(test2)

model.load_weights('my_model_w2v+bert.h5')
prediction = model.predict([test1, test2, test_left_words, test_right_words, test_left_cls, test_right_cls])